In [1]:
from transformers import CLIPProcessor, CLIPModel
import torch
from PIL import Image

In [2]:
# Load model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [3]:
# Load image and text
image = Image.open("example.jpg")  # replace with your image path
text = "A photo of a cat"

# Preprocess and encode
inputs = processor(text=[text], images=image, return_tensors="pt", padding=True)
outputs = model(**inputs)

# Get embeddings
image_embedding = outputs.image_embeds[0].detach().numpy()
text_embedding = outputs.text_embeds[0].detach().numpy()

In [4]:
# Calculate cosine similarity
def cosine_similarity(a, b):
    return torch.nn.functional.cosine_similarity(
        torch.tensor(a), torch.tensor(b), dim=0
    ).item()

similarity = cosine_similarity(image_embedding, text_embedding)
print(f"Cosine similarity between image and text: {similarity:.4f}")

Cosine similarity between image and text: 0.2733


In [ ]:
import chromadb
from chromadb.utils import embedding_functions

# Create a ChromaDB client and collection
client = chromadb.Client()
collection = client.create_collection(name="clip_embeddings")

# Optionally define a custom embedding function (not needed here, we use precomputed)
# Insert items
collection.add(
    documents=["A photo of a cat"],
    embeddings=[text_embedding.tolist()],  # convert numpy array to list
    ids=["text1"]
)

collection.add(
    documents=["example.jpg"],  # you can use the filename or any identifier
    embeddings=[image_embedding.tolist()],
    ids=["image1"]
)

In [ ]:
# Query similar embeddings
results = collection.query(
    query_embeddings=[text_embedding.tolist()],
    n_results=3
)

print(results)

{'ids': [['text1', 'image1']], 'embeddings': None, 'documents': [['A photo of a cat', 'example.jpg']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[None, None]], 'distances': [[0.0, 1.4534668922424316]]}
